## Pt1: Get the Weather Description and Amount of Precipitation for Each City



#### Generate a new set of 1,500 random latitudes and longitudes.

In [61]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

In [62]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-70.000, high=70.000, size=1500)
lngs = np.random.uniform(low=-140.000, high=140.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 

#### Get the nearest city using the citipy module.

In [63]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [64]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

744

#### Perform an API call with the OpenWeatherMap.

In [65]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_keys

In [66]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_keys
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=8ddc114b26221886571a6d9338b01791


In [67]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "San Francisco"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=8ddc114b26221886571a6d9338b01791&q=San Francisco



#### Make a Request for Data to an API

In [68]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "San Francisco"
city_weather = requests.get(city_url)
city_weather

<Response [200]>

In [69]:
# Make a 'Get' request for the city weather. Use the get() HTTP method 
city_weather.status_code

200

In [70]:
# Get the JSON format of the request; Get data from a response    
city_weather.json()

{'coord': {'lon': -76.88, 'lat': 1.17},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'model',
 'main': {'temp': 52.84,
  'pressure': 1011,
  'humidity': 97,
  'temp_min': 52.84,
  'temp_max': 52.84,
  'sea_level': 1011,
  'grnd_level': 753},
 'wind': {'speed': 0.83, 'deg': 253},
 'clouds': {'all': 100},
 'dt': 1573275046,
 'sys': {'country': 'CO', 'sunrise': 1573210152, 'sunset': 1573253601},
 'timezone': -18000,
 'id': 3669881,
 'name': 'San Francisco',
 'cod': 200}

In [71]:
# Get information from the JSON formatted data.
San_Francisco_data = city_weather.json()

In [72]:
# Retrieve data: Latitude and longitude, Maximum temperature, Percent humidity, Percent cloudiness, Wind speed, 
# Weather description(e.g.clouds, fog, light rain, clear sky)

lat = San_Francisco_data["coord"]["lat"]
lng = San_Francisco_data["coord"]["lon"]
max_temp = San_Francisco_data["main"]["temp_max"]
humidity = San_Francisco_data["main"]["humidity"]
clouds = San_Francisco_data["clouds"]["all"]
wind = San_Francisco_data["wind"]["speed"]
weather_description = San_Francisco_data['weather'][0]['description']
#rain_inches = San_Francisco_data['rain']['3h']
#snow_inches = San_Francisco_data['snow']['3h']
print(lat, lng, max_temp, humidity, clouds, wind, weather_description) #rain_inches, snow_inches)

1.17 -76.88 52.84 97 100 0.83 overcast clouds



#### Handle Request Errors

###### Evaluate whether the status code is equal to 200 and is therefore a valid response

In [73]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "San Francisco"
city_weather = requests.get(city_url)
if city_weather.status_code == 200:
    print(f"City Weather found.")
else:
    print(f"City weather not found.")
    
print(city_url)

City Weather found.
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=8ddc114b26221886571a6d9338b01791&q=San Francisco



#### Get the amount of rainfall in inches for the last three hours

In [74]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1       
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        
        
    # Using a try-except block, if it is raining,snowing get the amount of rainfall or snow in inches for the last three hours. 
    # If it is not raining, add 0 inches for the city.
        
        try:
            city_rain_inches = city_weather['rain']['3h']
        except(KeyError):
            city_rain_inches = 0
        try:
            city_snow_inches = city_weather['snow']['3h']
        except(KeyError):
            city_snow_inches = 0
            
            
         # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Weather Description":city_weather_description,
                          "Rain Inches(last 3hrs)":city_rain_inches,
                          "Snow Inches(last 3hrs)":city_snow_inches,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | jamestown
Processing Record 2 of Set 1 | taoudenni
Processing Record 3 of Set 1 | kharp
Processing Record 4 of Set 1 | ancud
Processing Record 5 of Set 1 | harrisonville
Processing Record 6 of Set 1 | bukachacha
Processing Record 7 of Set 1 | chuy
Processing Record 8 of Set 1 | lagoa
Processing Record 9 of Set 1 | rock springs
Processing Record 10 of Set 1 | busselton
Processing Record 11 of Set 1 | punta arenas
Processing Record 12 of Set 1 | lodja
Processing Record 13 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 14 of Set 1 | faya
Processing Record 15 of Set 1 | hithadhoo
Processing Record 16 of Set 1 | rawson
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | chabahar
Processing Record 19 of Set 1 | atuona
Processing Record 20 of Set 1 | pangnirtung
Processing Record 21 of Set 1 | port elizabeth
Processing Record 22 of Set 1 | olbia
Processing Record 

Processing Record 43 of Set 4 | warqla
City not found. Skipping...
Processing Record 44 of Set 4 | bengkulu
City not found. Skipping...
Processing Record 45 of Set 4 | fenoarivo
Processing Record 46 of Set 4 | arcachon
Processing Record 47 of Set 4 | cidreira
Processing Record 48 of Set 4 | georgetown
Processing Record 49 of Set 4 | lebu
Processing Record 50 of Set 4 | geraldton
Processing Record 1 of Set 5 | aranos
Processing Record 2 of Set 5 | namibe
Processing Record 3 of Set 5 | manchar
Processing Record 4 of Set 5 | buraydah
Processing Record 5 of Set 5 | balykshi
City not found. Skipping...
Processing Record 6 of Set 5 | terekli-mekteb
Processing Record 7 of Set 5 | san angelo
Processing Record 8 of Set 5 | coari
Processing Record 9 of Set 5 | nguiu
City not found. Skipping...
Processing Record 10 of Set 5 | plettenberg bay
Processing Record 11 of Set 5 | vestmanna
Processing Record 12 of Set 5 | hounde
Processing Record 13 of Set 5 | florida
Processing Record 14 of Set 5 | pray

Processing Record 30 of Set 8 | gelemso
Processing Record 31 of Set 8 | saint-francois
Processing Record 32 of Set 8 | miranda
Processing Record 33 of Set 8 | bur gabo
City not found. Skipping...
Processing Record 34 of Set 8 | millau
Processing Record 35 of Set 8 | xiuyan
Processing Record 36 of Set 8 | beaverlodge
Processing Record 37 of Set 8 | praia
Processing Record 38 of Set 8 | port-cartier
Processing Record 39 of Set 8 | bonthe
Processing Record 40 of Set 8 | punta de piedra
Processing Record 41 of Set 8 | sorvag
City not found. Skipping...
Processing Record 42 of Set 8 | santa marta
Processing Record 43 of Set 8 | esmeralda
Processing Record 44 of Set 8 | koratla
Processing Record 45 of Set 8 | anamur
Processing Record 46 of Set 8 | marystown
Processing Record 47 of Set 8 | toliary
City not found. Skipping...
Processing Record 48 of Set 8 | shelburne
Processing Record 49 of Set 8 | barawe
City not found. Skipping...
Processing Record 50 of Set 8 | tome-acu
City not found. Skip

Processing Record 12 of Set 12 | salinopolis
Processing Record 13 of Set 12 | calabozo
Processing Record 14 of Set 12 | mbacke
City not found. Skipping...
Processing Record 15 of Set 12 | saint-pierre
Processing Record 16 of Set 12 | praia da vitoria
Processing Record 17 of Set 12 | nizhniy kuranakh
Processing Record 18 of Set 12 | siuna
Processing Record 19 of Set 12 | gohana
Processing Record 20 of Set 12 | kodino
Processing Record 21 of Set 12 | pinega
Processing Record 22 of Set 12 | manzanillo
Processing Record 23 of Set 12 | negara
Processing Record 24 of Set 12 | alta floresta
Processing Record 25 of Set 12 | laurel
Processing Record 26 of Set 12 | revda
Processing Record 27 of Set 12 | khilok
Processing Record 28 of Set 12 | bonavista
Processing Record 29 of Set 12 | berdigestyakh
Processing Record 30 of Set 12 | usoke
Processing Record 31 of Set 12 | ketchikan
Processing Record 32 of Set 12 | saint anthony
Processing Record 33 of Set 12 | paslek
Processing Record 34 of Set 12 

#### Add the data to a new DataFrame.

In [75]:
# Convert a list of dictionaries to a Pandas DataFrame using df = pd.DataFrame(list with dictionaries).
# Convert the array of dictionaries to a Pandas DataFrame.
weather_database_df = pd.DataFrame(city_data)
weather_database_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Rain Inches(last 3hrs),Snow Inches(last 3hrs),Date
0,Jamestown,-33.21,138.60,63.19,41,0,13.42,AU,clear sky,0.0,0.0,2019-11-09 04:50:47
1,Taoudenni,22.68,-3.98,68.18,26,100,7.05,ML,overcast clouds,0.0,0.0,2019-11-09 04:50:47
2,Kharp,66.82,65.80,-1.83,92,78,10.54,RU,broken clouds,0.0,0.0,2019-11-09 04:50:47
3,Ancud,-41.87,-73.83,48.20,81,51,9.17,CL,broken clouds,0.0,0.0,2019-11-09 04:50:47
4,Harrisonville,38.65,-94.35,39.20,64,1,3.36,US,clear sky,0.0,0.0,2019-11-09 04:48:45
5,Bukachacha,52.99,116.91,29.23,69,0,1.88,RU,clear sky,0.0,0.0,2019-11-09 04:50:48
6,Chuy,-33.69,-53.46,61.14,94,2,7.67,UY,clear sky,0.0,0.0,2019-11-09 04:50:48
7,Lagoa,37.14,-8.45,51.01,76,27,3.36,PT,scattered clouds,0.0,0.0,2019-11-09 04:50:48
8,Rock Springs,41.59,-109.20,42.80,35,1,14.99,US,clear sky,0.0,0.0,2019-11-09 04:48:22
9,Busselton,-33.64,115.35,93.99,17,42,4.32,AU,scattered clouds,0.0,0.0,2019-11-09 04:47:49


In [76]:
# Reorder the columns as City, Country, Date, Lat, Lng, Max Temp, Humidity, Cloudiness, and Wind Speed, Weather Description,
# Rain inches(last 3hrs), Snow inches(last 3 hrs)

new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 
                    'Weather Description','Rain Inches(last 3hrs)', 'Snow Inches(last 3hrs)']

# Assign a new or the same DataFrame with new column order: df = df[new_column_order]
weather_database_df = weather_database_df[new_column_order]
weather_database_df.head(15)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain Inches(last 3hrs),Snow Inches(last 3hrs)
0,Jamestown,AU,2019-11-09 04:50:47,-33.21,138.60,63.19,41,0,13.42,clear sky,0.00,0.0
1,Taoudenni,ML,2019-11-09 04:50:47,22.68,-3.98,68.18,26,100,7.05,overcast clouds,0.00,0.0
2,Kharp,RU,2019-11-09 04:50:47,66.82,65.80,-1.83,92,78,10.54,broken clouds,0.00,0.0
3,Ancud,CL,2019-11-09 04:50:47,-41.87,-73.83,48.20,81,51,9.17,broken clouds,0.00,0.0
4,Harrisonville,US,2019-11-09 04:48:45,38.65,-94.35,39.20,64,1,3.36,clear sky,0.00,0.0
5,Bukachacha,RU,2019-11-09 04:50:48,52.99,116.91,29.23,69,0,1.88,clear sky,0.00,0.0
6,Chuy,UY,2019-11-09 04:50:48,-33.69,-53.46,61.14,94,2,7.67,clear sky,0.00,0.0
7,Lagoa,PT,2019-11-09 04:50:48,37.14,-8.45,51.01,76,27,3.36,scattered clouds,0.00,0.0
8,Rock Springs,US,2019-11-09 04:48:22,41.59,-109.20,42.80,35,1,14.99,clear sky,0.00,0.0
9,Busselton,AU,2019-11-09 04:47:49,-33.64,115.35,93.99,17,42,4.32,scattered clouds,0.00,0.0


In [77]:
# Save the new DF as a CSV file or Create the output file (CSV).

output_data_file = "weather_data/WeatherPy_challenge.csv"
# Export the City_Data into a CSV.
weather_database_df.to_csv(output_data_file, index_label="City_ID")

In [82]:
#Answer this question using Pandas methods: How many cities have recorded snow?
snow_record= weather_database_df.loc[weather_database_df["Snow Inches(last 3hrs)"] > 0]
len(snow_record)

15

In [83]:
rain_record= weather_database_df.loc[weather_database_df["Rain Inches(last 3hrs)"] > 0]
len(rain_record)

61

In [84]:
snow_record

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain Inches(last 3hrs),Snow Inches(last 3hrs)
59,Lensk,RU,2019-11-09 04:51:02,60.71,114.91,30.45,96,100,9.89,light snow,0.0,0.75
109,Balakhta,RU,2019-11-09 04:51:14,55.38,91.62,25.07,77,100,17.69,light snow,0.0,1.12
195,Vestmanna,FO,2019-11-09 04:51:32,62.16,-7.17,39.20,75,100,18.34,light snow,0.0,0.19
212,Turukhansk,RU,2019-11-09 04:51:35,65.80,87.96,-16.59,95,100,7.11,light snow,0.0,0.13
241,Parry Sound,CA,2019-11-09 04:51:41,45.34,-80.04,32.00,84,55,10.00,light snow,0.0,0.63
261,Khatanga,RU,2019-11-09 04:51:45,71.98,102.47,-12.79,94,100,15.28,light snow,0.0,0.19
299,Meadow Lake,CA,2019-11-09 04:51:52,54.13,-108.44,18.41,95,100,6.80,light snow,0.0,0.19
312,Gimli,CA,2019-11-09 04:51:56,50.63,-96.99,26.01,95,80,4.00,broken clouds,0.0,0.06
375,Brae,GB,2019-11-09 04:52:11,60.40,-1.35,37.40,59,20,3.36,light snow,0.0,0.13
381,Yerbogachen,RU,2019-11-09 04:52:12,61.28,108.01,29.62,85,100,14.41,light snow,0.0,0.31


In [85]:
rain_record

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain Inches(last 3hrs),Snow Inches(last 3hrs)
13,Hithadhoo,MV,2019-11-09 04:50:50,-0.60,73.08,81.36,77,26,8.97,light rain,2.94,0.0
31,Uvinza,TZ,2019-11-09 04:50:55,-5.10,30.39,77.68,60,49,5.06,light rain,0.50,0.0
34,Caarapo,BR,2019-11-09 04:50:56,-22.63,-54.82,71.38,91,88,4.90,light rain,1.00,0.0
47,Padang,ID,2019-11-09 04:50:59,-0.92,100.36,78.24,81,97,3.94,light rain,2.38,0.0
108,Arraial Do Cabo,BR,2019-11-09 04:51:14,-22.97,-42.02,72.70,94,100,6.46,light rain,0.44,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
651,Tuensang,IN,2019-11-09 04:53:13,26.24,94.81,61.66,78,100,0.38,light rain,1.12,0.0
656,Lumeje,AO,2019-11-09 04:53:14,-11.56,20.78,73.36,70,0,5.84,light rain,0.44,0.0
665,Codajas,BR,2019-11-09 04:53:15,-3.84,-62.06,74.64,94,100,0.60,light rain,0.25,0.0
666,Barra Patuca,HN,2019-11-09 04:53:16,15.80,-84.28,82.58,76,4,11.34,light rain,2.44,0.0
